In [3]:
import pandas as pd 

In [26]:
#file header row count
header_size = 0

#location tuples
locations = [(51.54052,5.85307),(52.296556,4.510817),(52.803657,5.050509),(52.13795,4.83819),(52.111621,5.708419),(52.875725,6.932432)]
#station names
stations = ["NL10131","NL10444","NL10538","NL10633","NL10738","NL10929"]

for i in range(len(stations)):
    s = stations[i]
    l = locations[i]					

    df_2012_2019 = []
    for year in ["2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021"]: 
        #expects a directory structure data/2021
        NH3 = pd.read_csv(f"data/{year}_NH3.csv", sep=';', skiprows=header_size)
        NH3.rename(columns={"Begindatumtijd":"Datetime", s:"NH3"}, inplace=True)

                
        #time date conversion
        # 20210101 00:00 -> 2021/01/01 00:00
        # step 1, Convert second column to datetime (20210101 00:00 -> 2021/01/01 00:00)
        NH3["Datetime"] = pd.to_datetime(NH3["Datetime"], format='%Y%m%d %H:%M')
        df = NH3
        df.set_index("Datetime", inplace=True)
    

        try:
            ###filter outliers
            #find IQR
            Q1, Q3= df["NH3"].quantile(0.25), df["NH3"].quantile(0.75)
            IQR = Q3-Q1

            #remove outliers based on the "IQR * 1.5" rule
            df = df[(df["NH3"]  > Q1 - (IQR * 1.5)) & (df["NH3"] < Q3 + (IQR * 1.5))]

            ###interpolate
            # resample data to daily frequency and calculate median values {H, D, W, M, Q, Y}
            df = df['NH3'].resample("H").median(numeric_only=True).interpolate()

            ###normalize min max
            normalized_df = (df-df.min())/(df.max()-df.min())

            df_2012_2019.append(normalized_df)
        except KeyError:
            print("KeyError")


        #concat row wise
        #df = pd.DataFrame.from_records(df_2012_2019)
        df  = pd.concat(df_2012_2019)
        df = df.to_frame()

        #static columns
        df["Lat"] = l[0]
        df["Lon"] = l[1]
        df["Station"] = s
        #print(df.head(10))
    
        df.to_csv(f"measurements/normalized/{s}_2012_2021.csv", index=True)

KeyError
